# Persistence and ORMs

## Databases

90% of applications have a need to store data. Storing data simply in files becomes difficult and inefficient after a while. After a while the file containing the data might start getting bigger, and so you would need to start implementing features to speed up the file reading and writing, which in a nutshell is just what the databases do.

Databases are purpose built applications for storing and retrieving data.

## CAP theorem

CAP theorem applies to distributed data stores. According to CAP, a distributed database can only guarantee 2 of these traits:

```mermaid
mindmap
  root(CAP)
    Consistency
    Availability
    Partition tolerance
```

Consistency - all reads receive the most recently written data.
Availability - database is always ready for use.
Partition tolerance - database can continue to function properly even if there are networking issues between nodes.

## Migrations

```mermaid
mindmap
  root(Database migrations)
    Database first
    Code first
    Model first
```

### Benefits of using migrations

Pros: 
- Speeds up the development, since you can generate part of reflected code from the existing code.
- Ensures consistency between database and code.
- Allows to change either the code or database easier.

Cons:
- If it does not work properly out of the box, then it might end up being a huge time waste.

### Database first

Database first Migrations is a development approach where you design your database structure first. It can be done using tools like MySQL Workbench or SQL Server Management Studio, or just running queries via CLI. The point is that the database schema must be created before you start generating code. Once the database is available, then you generate the code, such as entity classes, based on the database schema created. This approach is particularly useful for projects where you need to work with an existing database or want to ensure your code aligns with a predefined database structure.

### Model first

Model-First Migrations is a development approach where you design your database schema using a visual modeling tool, designer tool within an ORM framework like Entity Framework, or some intermediate modeling language.

After creating the model, you generate the corresponding database schema and code, including entity classes and relationships. This approach is OK when you first start out by designing your database visually.

### Code first

Code first migrations allows to generate database structure according to the code that is written. It is a typical feature of ORM frameworks like Entity Framework.

Pros:
- Fastest to work with.
- Changes are neatly versioned.
- Code always (mostly) work.

Cons:
- Tough luck if the migrations library does not support some database feature needed.


## ORM vs no ORM

ORM stands for object-relational-mapper. Using ORM means that you interact with the database via classes and methods instead of working directly with the queries against the database. ORMs simplifies the usage of databases at the cost of reduced flexibility.

Pros of using ORM:
- No need to deal with SQL query statements.
- Easier to refactor code, since IDE can track down all the references.
- ORM takes care of lots of boilerplate code.

Cons of using ORM:
- You can only use the database features that the ORM framework supports.
- Since query statements are generated by ORM, they can be suboptimal performance wise.

## Connection strings

Connection strings specify how to connect and use the data source.

Example connection string:
>`User ID=root;Password=myPassword;Host=localhost;Port=5432;Database=myDataBase;Pooling=true;Min Pool Size=0;Max Pool Size=100;Connection Lifetime=0;`

Connection strings can contain usernames, passwords, host address and various other parameters.

Connections strings differ based on database connection driver used. Same connection string that worked for one relational database, may not work for other relation database. Always check the reference like https://www.connectionstrings.com/ to be sure how to assemble connection string correctly for your case.

## Persistence in .NET

.NET has libraries to work with most of the commonly available databases. This section will take a look at 2 libraries for working with relational databases: `Dapper` and `EntityFramework`.

For these examples consider there exists a database available with schema like this:

```sql
CREATE TABLE courses (
    id INT PRIMARY KEY AUTO_INCREMENT,
    course_name VARCHAR(100) NOT NULL,
    course_description TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE subjects (
    id INT PRIMARY KEY AUTO_INCREMENT,
    subject_name VARCHAR(100) NOT NULL,
    subject_description TEXT,
    course_id INT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (course_id) REFERENCES courses(id) ON DELETE CASCADE
);
```

### Dapper

In [ ]:
// Define the classes that represent the schema:

public class Course
{
    public int Id { get; set; }
    public string CourseName { get; set; }
    public string CourseDescription { get; set; }
    public DateTime CreatedAt { get; set; }
}

public class Subject
{
    public int Id { get; set; }
    public string SubjectName { get; set; }
    public string SubjectDescription { get; set; }
    public int CourseId { get; set; }
    public DateTime CreatedAt { get; set; }
}

Add the `Dapper` package from nuget:

In [ ]:
#r "nuget:Dapper"
#r "nuget:MySqlConnector"

Define the connection string, provide the correct credentials:

In [ ]:
// In this example the MySQL database is used
var connectionString = "Server=localhost;Database=example;Uid=root;Pwd=password;";

In [ ]:
using MySqlConnector;

var connection = new MySqlConnection(connectionString);

In [ ]:
using Dapper;

var insertCourse = @"INSERT INTO Courses 
    (course_name, course_description, created_at) 
    VALUES (@CourseName, @CourseDescription, @CreatedAt);
    SELECT LAST_INSERT_ID();";

var courseId = connection.Query<int>(
    insertCourse, 
    new 
    { 
        CourseName = "Course 1", 
        CourseDescription = "Course 1 Description", 
        CreatedAt = DateTime.Now,
    });

var insertSubject = @"INSERT INTO Subjects 
    (subject_name, subject_description, course_id, created_at) 
    VALUES (@SubjectName, @SubjectDescription, @CourseId, @CreatedAt);";

connection.Query(
    insertSubject, 
    new 
    { 
        SubjectName = "Subject 1",
        SubjectDescription = "Subject 1 Description",
        CourseId = courseId,
        CreatedAt = DateTime.Now 
    });

In [ ]:
var selectCourses = "SELECT Id, course_name as CourseName, course_description as CourseDescription, created_at as CreatedAt FROM Courses;";

var courses = connection.Query<Course>(selectCourses);

courses.DisplayTable();

### Entity Framework

In Entity Framework everything is going to revolve around the `DbContext` class and `DbSet`s defined in it.

In [ ]:
#r "nuget:Microsoft.EntityFrameworkCore"

// A MySqlConnector adapter for Entity Framework
#r "nuget:Pomelo.EntityFrameworkCore.MySql"

In [ ]:
public class Course
{
    public int Id { get; set; }
    public string CourseName { get; set; }
    public string CourseDescription { get; set; }
    public DateTime CreatedAt { get; set; }

    // Relations will be resolved by EF Core, so define them explicitly here
    public List<Subject> Subjects { get; set; }
}

public class Subject
{
    public int Id { get; set; }
    public string SubjectName { get; set; }
    public string SubjectDescription { get; set; }
    public DateTime CreatedAt { get; set; }

    // Defining the explicit relations
    public int CourseId { get; set; }
    public Course Course { get; set; }
}

In [ ]:
using Microsoft.EntityFrameworkCore;

public class StudiesContext : DbContext
{
    public DbSet<Course> Courses { get; set; }
    public DbSet<Subject> Subjects { get; set; }

    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
    {
        // Should be configurable in the ideal case
        var connectionString = "Server=localhost;Database=example;Uid=root;Pwd=password;";

        optionsBuilder.UseMySql(
            connectionString, 
            // There are syntax differences between MySQL versions
            ServerVersion.AutoDetect(connectionString)
        );
    }

    protected override void OnModelCreating(ModelBuilder modelBuilder)
    {
        // In this example the configuration is very verbose
        // but if the naming convention were to be followed, most of it could be ommited.

        // Also if the code first migration were used, then configuration would mostly not be needed

        var courseConfig = modelBuilder.Entity<Course>();
        courseConfig.ToTable("courses");
        courseConfig.HasKey(c => c.Id);
        courseConfig.Property(c => c.CourseName)
            .HasColumnName("course_name")
            .HasMaxLength(100);
        courseConfig.Property(c => c.CourseDescription)
            .HasColumnName("course_description");
        courseConfig.Property(c => c.CreatedAt)
            .HasColumnName("created_at");

        var subjectConfig = modelBuilder.Entity<Subject>();
        subjectConfig.ToTable("subjects");
        subjectConfig.HasKey(s => s.Id);
        subjectConfig.Property(s => s.SubjectName)
            .HasColumnName("subject_name")
            .HasMaxLength(100);
        subjectConfig.Property(s => s.SubjectDescription)
            .HasColumnName("subject_description");
        subjectConfig.Property(s => s.CreatedAt)
            .HasColumnName("created_at");
        subjectConfig.Property(s => s.CourseId)
            .HasColumnName("course_id");
    
        subjectConfig
            .HasOne(s => s.Course)
            .WithMany(c => c.Subjects)
            .HasForeignKey(s => s.CourseId)
            .OnDelete(DeleteBehavior.Cascade);
    }
}

In [ ]:
var dbContext = new StudiesContext();

var course = new Course
{
    CourseName = "Course 2",
    CourseDescription = "Course 2 Description",
    CreatedAt = DateTime.Now,
    Subjects = new List<Subject>
    {
        new Subject
        {
            SubjectName = "Subject 2",
            SubjectDescription = "Subject 2 Description",
            CreatedAt = DateTime.Now
        }
    }
};

dbContext.Courses.Add(course);
dbContext.SaveChanges();

In [ ]:
var courses = dbContext.Courses.ToList();
courses.DisplayTable();

In [ ]:
var subjects = dbContext.Subjects.ToList();
subjects.DisplayTable();

In [ ]:
var courseWithSubject = dbContext.Courses
    .Include(c => c.Subjects)
    .FirstOrDefault();

courseWithSubject.Display();

### Code first migration in Entity Framework

CLI tool (dotnet tool) will have to be installed to start generating migrations:

```
dotnet tool install --global dotnet-ef
```

After the tool is installed, run the command to generate migrations:

```
dotnet ef migrations add <name> --project <path to project> --startup-project <path to startup projects> --output-dir <relative path in project dir where to place generated file>
```

After the migration is generated it can be applied by:

a) Scripting the migration and applying it manually: `dotnet ef migrations script`.

b) Applying it by launching the project: `dotnet ef database update`.

c) Applying it automatically via code on project startup: `dbContext.Database.Migrate();`


## Testing code with databases

There are several general approaches how this could be done:

- In general libraries like `EntityFramework` have In Memory adapters, which stores the data in memory and emulates the usage of real database with having to use one. Pros of such approach is that it is very easy to set up. Cons is that In Memory adapter tries to emulate the real database, but not always succeeds. This failure to correctly replicate the usage of database, can lead to false positives, where tests succeed, but the actual implementation doesn't work. This can lead to false negatives as well, where tests fail, while the implementation is correct.
- Using Sqlite as database for testing. Sqlite is an "In Application" database that stores all the data in file and, instead of relying on external service for the database engine, can run everything needed in the hosting application. This is a better representation of the actual database and would be more robust approach than using In Memory database. However the problem is still that in real environment different database engine that Sqlite can be used and this would make tests not fully representative of the final solution.
- Using test containers.

### Testing with test containers

Test containers is a very good option to setup the database for integration tests. It depends on having docker service running on the machine.

Test containers launch a docker container with the desired database. After the test completes, the database container is removed. This allows to run tests against the actual database that is launched and cleared only for the specific test.

Although setting up the test container involves quite some code to do that properly.

You can use the class below as fixture for `xUnit` framework, it assumes that tested project is an ASP.NET application and code first migrations are used.

It depends on 2 packages installed:
- `Testcontainers.MySql`
- `Respawn`

```csharp
public class IntegrationTestFactory : WebApplicationFactory<Program>, IAsyncLifetime
{
    private const string DatabaseName = "db name";
    private const string DatabaseUser = "db user";
    private const string DatabasePassword = "user password";

    private readonly MySqlContainer _container = new MySqlBuilder()
        .WithImage("mysql:<version needed>")
        .WithUsername(DatabaseUser)
        .WithPassword(DatabasePassword)
        .WithDatabase(DatabaseName)
        .WithCleanUp(true)
        .Build();

    public IApplicationDbContext DbContext { get; private set; } = null!;
    private Respawner _respawner = null!;
    private DbConnection _connection = null!;

    public async Task ResetDatabaseAsync()
    {
        await _respawner.ResetAsync(_connection);
    }

    public async Task InitializeAsync()
    {
        await _container.StartAsync();
        var dbContext = Services.GetRequiredService<ApplicationDbContext>();

        DbContext = dbContext;

        _connection = dbContext.Database.GetDbConnection();
        await _connection.OpenAsync();

        _respawner = await Respawner.CreateAsync(
            _connection,
            new RespawnerOptions
            {
                DbAdapter = DbAdapter.MySql,
                TablesToIgnore =
                [
                    new Table("__efmigrationshistory"),
                    new Table("__EFMigrationsHistory "),
                ],
            }
        );
    }

    public new async Task DisposeAsync()
    {
        await _container.DisposeAsync();
    }

    protected override void ConfigureWebHost(IWebHostBuilder builder)
    {
        var configValues = new Dictionary<string, string?>
        {
            ["Database:Host"] = "localhost",
            ["Database:Port"] = _container.GetMappedPublicPort(3306).ToString(), // default mysql port
            ["Database:User"] = DatabaseUser,
            ["Database:Password"] = DatabasePassword,
            ["Database:Database"] = DatabaseName,
        };
        var configuration = new ConfigurationBuilder().AddInMemoryCollection(configValues).Build();

        builder
            .UseConfiguration(configuration)
            .ConfigureAppConfiguration(configurationBuilder =>
            {
                configurationBuilder.AddInMemoryCollection(configValues);
            });
    }
}

[CollectionDefinition(nameof(DatabaseTestCollection))]
public class DatabaseTestCollection : ICollectionFixture<IntegrationTestFactory> { }
```

In the actual test class, this fixture should be used like this:

```csharp
[Collection(nameof(DatabaseTestCollection))]
public class MyIntegrationTest : IAsyncLifetime
{
    private readonly IApplicationDbContext _dbContext;
    private readonly Func<Task> _resetDatabase;

    public MyIntegrationTest(IntegrationTestFactory factory)
    {
        _dbContext = factory.DbContext;
        _resetDatabase = factory.ResetDatabaseAsync;
    }

    // All the tests can go here

    public Task DisposeAsync() => Task.CompletedTask;

    public Task InitializeAsync() => _resetDatabase();
}
```